# Sartorius - PyTorch Mask R-CNN infererence

## Libraries

In [ ]:
DEBUG = False
KAGGLE = False
COLAB = True

In [ ]:
from psutil import virtual_memory, cpu_count

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')
print('No of CPU cores:', cpu_count())

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if COLAB:
    !pip install git+https://github.com/albumentations-team/albumentations.git

In [ ]:
import os
import cv2
import time
import json
import random
import collections
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
import torchvision
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
if DEBUG:
    warnings.filterwarnings('ignore', category=UserWarning) 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' if KAGGLE else '0'
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')

## Configs

In [ ]:
VER = 'ver0'
WORK_DIR = '/content/drive/MyDrive/sartorius'
DATA_PATH = '../input/sartorius-cell-instance-segmentation' if KAGGLE else f'{WORK_DIR}/data'
MDLS_PATH = f'../input/sartorius-models-{VER}' if KAGGLE else f'{WORK_DIR}/models_{VER}'
with open(f'{MDLS_PATH}/config.json', 'r') as file:
    CONFIG = json.load(file)
print('config loaded:', CONFIG)
RESNET_MEAN = (.485, .456, .406)
RESNET_STD = (.229, .224, .225)

def optimal_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus * 4) if num_gpus else (num_cpus - 1)
    print('optimal number of workers is', optimal_value)
    return optimal_value

start_time = time.time()

## Datasets and loaders

In [ ]:
class CellInferDataset(Dataset):
    def __init__(self, img_dir, aug=None):
        self.aug = aug
        self.img_dir = img_dir
        self.img_idxs = [x[:-4] for x in os.listdir(self.img_dir)]
    
    def __getitem__(self, idx):
        img_idx = self.img_idxs[idx]
        img_path = os.path.join(self.img_dir, img_idx + '.png')
        img = Image.open(img_path).convert('RGB')
        if self.aug:
            img = np.array(img).astype(np.float32) / 255
            augmented = self.aug(image=img)
            img = augmented['image']
        return {'image': img, 'image_id': img_idx}

    def __len__(self):
        return len(self.img_idxs)

## Model

In [ ]:
def cell_model(pretrained=True, pretrained_backbone=True):
    N_CLASSES = 2
    if CONFIG['normalize']:
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(
            pretrained=pretrained, 
            pretrained_backbone=pretrained_backbone,
            box_detections_per_img=CONFIG['num_boxes'],
            image_mean=RESNET_MEAN, 
            image_std=RESNET_STD
        )
    else:
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(
            pretrained=pretrained,
            pretrained_backbone=pretrained_backbone,
            box_detections_per_img=CONFIG['num_boxes']
        )
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, N_CLASSES)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, 
        hidden_layer, 
        N_CLASSES
    )
    return model

## Utilities

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

## Inference

In [ ]:
with open(f'{MDLS_PATH}/modelfiles.json', 'r') as file:
    modelfiles = json.load(file)
print(modelfiles)

In [ ]:
with open(f'{MDLS_PATH}/modelfiles.json', 'r') as file:
    modelfiles = json.load(file)
print(modelfiles)

models = []
for model_file in modelfiles:
    model = cell_model(pretrained=False, pretrained_backbone=False)
    model.to(DEVICE)
    checkpoint = torch.load(f'{MDLS_PATH}/{model_file.split("/")[-1]}')
    model.load_state_dict(checkpoint['model_state_dict'])
    models.append(model)
    print('model loaded:', model_file)

In [ ]:
test_aug =  A.Compose([ToTensorV2(p=1)])

subm_dataset = CellInferDataset(
    f'{DATA_PATH}/test', 
    aug=test_aug)

In [ ]:
SCORE_TH = 0
SINGLE_MASK_TH = CONFIG['mask_th']
IMG_MASK_TH = 0
subm = []
for n_sample, sample in enumerate(subm_dataset):
    img = sample['image']
    img_id = sample['image_id']
    img_mask = np.zeros((CONFIG['height'], CONFIG['width']))
    for model in models[:1]:
        model.eval()
        with torch.no_grad():
            pred = model([img.to(DEVICE)])[0]
        for i, mask in enumerate(pred['masks']):
            score = pred['scores'][i].cpu().item()
            if score < SCORE_TH:
                continue
            mask = mask.cpu().numpy()
            mask = mask > SINGLE_MASK_TH
            img_mask += mask[0, ...] / (len(models))
    img_mask = img_mask > IMG_MASK_TH
    if n_sample < 4:
        plt.imshow(img.numpy().transpose((1, 2, 0)))
        plt.imshow(img_mask, alpha=.3)
        plt.title(f'image {n_sample} and mask')
        plt.axis('off')
        plt.show()
    rle = rle_encode_less_memory(img_mask)
    subm.append((img_id, rle))
    all_img_idxs = [img_id for img_id, rle in subm]
    if img_id not in all_img_idxs:
        subm.append((img_id, ''))
    
df_subm = pd.DataFrame(subm, columns=['id', 'predicted'])
df_subm.to_csv('submission.csv', index=False)
df_subm.head()